# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [36]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()


True

In [37]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [38]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [39]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      # block: {number: 74087200}
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
      lastTxHash
    }  
}
""")

trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")

stats_query = gql("""
query stats(
    $last_id: ID!
  ) {
    futuresStats(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      totalTrades
      totalVolume
      feesPaid
      pnl
      pnlWithFeesPaid
      liquidations
    }  
}
""")


### Run a query

In [40]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

position_bytes_cols = ['asset']

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, decimal_cols=position_cols, bytes_cols=position_bytes_cols)
df_position['netPnl'] = df_position['pnl'] - df_position['feesPaid']


In [41]:
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x0,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,77.199529739466510105,0,1.336,1.336,1.237228592486890451,NaN,0,...,0,0,False,True,3346.196453288167397377,7076.735400033458191233,642736.21082586136343291,0,0xb39d36479be01bed531b5a94363fcd31c33c51475386...,0
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108,0xb5d5683a5f43dc5fcfda20b4a44e642fed605665250e...,-5.808908608197160176
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,MATIC,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125,0x77e50f88ec5820b4327c68471cd1dc4aa4990ecf9935...,24.196620353360896054
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x100,0x2c23f12195077bee4ac4f7d598af30904a5fbf91,MATIC,98,0,1.271783672974139673,1.271783672974139673,1.268341831466371221,1.268341831466371221,6.767650966258712392,...,8.993264259656970653,-0.010044854998744103,False,False,97.764341851602997636,0,0,4994.617789850222396638,0x0ce56789360409b4db0cd3237277f9be446758e51a8c...,-2.225613293398258261
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x101,0xd8e80c06f9f3a41c85cad28117e01ebd50fd535e,MATIC,777.321876625415779658,0,1.257577965021184482,1.257577965021184482,1.205252195554925268,1.205252195554925268,210.365882266396886773,...,13.943417100998227914,-1.755761096539670812,False,False,973.988580694274767704,0,0,9901.343924544948537964,0xd7b5a651cc808bdd6543a3684ecb3b02e9c599f52a98...,196.422465165398658859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6970,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,XAG,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538,0xd077fc1cbcbee2fa9d0080e453eba181e9355e546261...,-23.708877311667551749
6971,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,XAG,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318,0x4241eafd00aed3106ac684b285fb282da348cc8a9af6...,-18.241242445271967120
6972,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,XAG,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407,0x52611516fd482b26c06642c425f0d5171e9f20997525...,625.441828012558800042
6973,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,XAG,183.427032240973901673,0,21.728166360389986619,21.728166360389986619,21.816099329108998862,21.816099329108998862,14.574525202046228121,...,5.081863514639043735,-0.794828372462505745,False,False,194.124865555918580315,0,0,7217.281605994306985757,0xb87037c300d30c0cd78d7b1516473d01c2e537fb4bd2...,9.492661687407184386


In [42]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = [
    'marketKey',
    'asset',
]

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)
df_trade['netPnl'] = df_trade['pnl'] - df_trade['feesPaid']


In [43]:
df_trade


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl
0,0x00028b72733b86a53c61e5ed7a842263945d64c6c813...,0x70b3d1ff337bcceeb3e2cf5b05efbffc50af87d7,1677087956,MATIC,sMATICPERP,DelayedOffchain,1.346733270313601043,1047.5773280909209386,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0xc4,9.397554354752353737,215.552868807478393106,6866.2022,0,True,206.155314452726039369
1,0x0003d5401b4c7a8a1feb9e8ea750336e6df42b5cd3ff...,0x50d36f27872f637ec9ecb4eedff300df44a608ab,1673640747,sETH,sETHPERP,DelayedOffchain,1420.509945340577467893,83.803545657223511638,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x26b,3.128026947594952567,-0.142862529594830699,-1.5882,0,True,-3.270889477189783266
2,0x000ad64e851c16a8d33b9dcfa80e94ae3952fe84223e...,0x6fb737d1ebb73cda6cfa36fd16d9273065d1b084,1676621493,OP,sOPPERP,DelayedOffchain,2.719976157048370677,98,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x10b,2.414277528497919551,0,-101.5395,-101.5395,False,-2.414277528497919551
3,0x000b5a721737f10af3ce29df92ed6783f4d6af7e2ef2...,0x9194efdf03174a804f3552f4f7b7a4bb74badb7f,1677529912,sETH,sETHPERP,DelayedOffchain,1620.712688734056254005,3391.674352536051959999,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xd90,32.481715964634636402,60.703857152210701221,-37.6152,-20.8516,False,28.222141187576064819
4,0x00119b365bda283f320327be36079df51865a5101433...,0x6fb737d1ebb73cda6cfa36fd16d9273065d1b084,1676867485,FTM,sFTMPERP,DelayedOffchain,0.523495113625592428,168.930189366378676981,0xc18f85a6dd3bcd0516a1ca08d3b1f0a4e191a2c4-0x36,2.224952011347595962,0,-286.474512677729026394,-1372.534845227671304419,False,-2.224952011347595962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17817,0xffd7c7fe1e4e726b4e93910453d6c07343dc28045f7e...,0x1bd477fc34b96ab561156f2438b8f9a18fcb5673,1676274577,sETH,sETHPERP,DelayedOffchain,1522.77873592749243167,251.247711634623038918,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x723,8.335012093830431712,0,4.160165849683939389,4.160165849683939389,False,-8.335012093830431712
17818,0xffe545b86e9a47ba061ddbe7f20d4729169f347009c1...,0x0c8aa570a1dfeee5258f3c13e2e967da24bbb505,1673262644,sETH,sETHPERP,DelayedOffchain,1322.79752629640388717,52.755917631435047066,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x190,1.137240243353251903,0.083020094148102862,0.2075,0,True,-1.054220149205149041
17819,0xffe55e5a83b9be19c56cbb873966c8fa586b66c24422...,0x182a36ad47ea21d96d864080b7baceda7ce44019,1676398634,sETH,sETHPERP,DelayedOffchain,1550.020923468874649187,2018.301296096519858795,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x7aa,4.445700514095363865,0,3.1557,15.9371,False,-4.445700514095363865
17820,0xfff96e959bc63b043de5c88eb69570a307f454257cf7...,0xc31b0df48319a83eeffd9280f3f29707ec950092,1671649414,sETH,sETHPERP,DelayedOffchain,1210.561005015049885549,49.922118336338367668,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x20,1.101808180521765695,0,0.0841,0.0841,False,-1.101808180521765695


In [44]:
stat_params = {
    'last_id': ''
}

stat_decimal_cols = [
    'pnl',
    'feesPaid',
    'pnlWithFeesPaid'
]

df_stat = await run_recursive_query(stats_query, stat_params, 'futuresStats')
df_stat = df_stat.replace({None: np.nan})
df_stat = clean_df(df_stat, decimal_cols=stat_decimal_cols)
df_stat


,id,account,totalTrades,totalVolume,feesPaid,pnl,pnlWithFeesPaid,liquidations
0,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,3,41653669514579068142700,536.199029591903898345,0,-534.199029591903898345,0
1,0x0028112a819a2fea8802b27c8abd755152f548ba,0x0028112a819a2fea8802b27c8abd755152f548ba,0,0,0,0,0,0
2,0x0045fa93dedb0af02956f7d7c306b34263688872,0x0045fa93dedb0af02956f7d7c306b34263688872,18,5415812020511898350970,43.86009579536806738,-13.050673405243633794,-54.910769200611701174,0
3,0x0046e6b678371f3576408873b3941b9ab10e3c72,0x0046e6b678371f3576408873b3941b9ab10e3c72,4,2243356863904652944833,14.328902650518394115,0.158035901794500817,-14.170866748723893298,0
4,0x0099560ec05a765a6b95443d4039ce1daf24f805,0x0099560ec05a765a6b95443d4039ce1daf24f805,3,51999128887105781261841,69.211006162079670803,0,-67.211006162079670803,0
...,...,...,...,...,...,...,...,...
1753,0xff6f84ce85166396d1402d14721ac350634a5d4b,0xff6f84ce85166396d1402d14721ac350634a5d4b,3,878375049354077615935,6.684343934204460865,1.208595124603055855,-3.47574880960140501,0
1754,0xff714698695caab8c83061bb9732967a50730570,0xff714698695caab8c83061bb9732967a50730570,2,299851768801504608411,2.224866322280623873,-0.091948537070266261,-2.316814859350890134,0
1755,0xff789ab6730c22da1e095645836cb70eaf5cfc51,0xff789ab6730c22da1e095645836cb70eaf5cfc51,0,0,0,0,0,0
1756,0xffd3003a187d9951e49bda206f32a6e765360427,0xffd3003a187d9951e49bda206f32a6e765360427,0,0,0,0,0,0


## Trades vs Positions

In [45]:
trade_pnl = df_trade.groupby('positionId')[['pnl', 'netPnl']].sum().reset_index()

df_pnl = df_position[['id', 'pnl', 'netPnl', 'asset', 'isLiquidated', 'isOpen']].merge(trade_pnl, left_on='id', right_on='positionId', suffixes=['_position', '_trade'])
df_pnl['diff'] = df_pnl['netPnl_trade'] - df_pnl['netPnl_position']
df_pnl



,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,2.052092628903229758,-5.808908608197160176,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,2.052092628903229758,-5.808908608197160176,0E-18
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,29.769574101247450474,24.196620353360896054,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,29.769574101247450474,24.196620353360896054,0E-18
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x100,6.767650966258712392,-2.225613293398258261,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x100,6.767650966258712392,-2.225613293398258261,0E-18
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x101,210.365882266396886773,196.422465165398658859,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x101,210.365882266396886773,196.422465165398658859,0E-18
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x102,-55.277651493320571662,-76.892380216676382901,MATIC,False,True,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x102,-55.277651493320571662,-76.892380216676382901,0E-18
...,...,...,...,...,...,...,...,...,...,...
6947,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,-15.956808702549099059,-23.708877311667551749,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,-15.956808702549099059,-23.708877311667551749,0E-18
6948,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,-13.642729241618423108,-18.241242445271967120,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,-13.642729241618423108,-18.241242445271967120,0E-18
6949,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,641.506744317628356061,625.441828012558800042,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,641.506744317628356061,625.441828012558800042,0E-18
6950,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,14.574525202046228121,9.492661687407184386,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,14.574525202046228121,9.492661687407184386,0E-18


In [46]:
df_pnl.loc[(df_pnl['netPnl_position'] != df_pnl['netPnl_trade']), :].sort_values('diff', ascending=True)

,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff


In [47]:
df_pnl.loc[(df_pnl['pnl_position'] != df_pnl['pnl_trade']), :].sort_values('diff', ascending=True)

,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff


## Compare position and trader pnl

In [48]:
df_position_agg = df_position.groupby('account')[['pnl', 'pnlWithFeesPaid']].sum().reset_index()
df_trade_agg = df_trade.groupby('account')[['pnl']].sum().reset_index()

df_compare = df_stat[['account', 'pnl', 'pnlWithFeesPaid', 'liquidations']].merge(df_position_agg, on='account', suffixes=['_stat', '_position'])
df_compare = df_compare.merge(df_trade_agg, on='account', suffixes=['', '_trade'])

In [49]:
df_compare

,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl
0,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,0,-534.199029591903898345,0,0,-534.199029591903898345,0
1,0x0045fa93dedb0af02956f7d7c306b34263688872,-13.050673405243633794,-54.910769200611701174,0,-13.050673405243633794,-40.910769200611701174,-13.050673405243633794
2,0x0046e6b678371f3576408873b3941b9ab10e3c72,0.158035901794500817,-14.170866748723893298,0,0.158035901794500817,-14.170866748723893298,0.158035901794500817
3,0x0099560ec05a765a6b95443d4039ce1daf24f805,0,-67.211006162079670803,0,0,-67.211006162079670803,0
4,0x00a25c455636cc01bffcb717db900a6d4c767a8a,8086.502208045348417539,4267.026063338116950934,0,8086.502208045348417539,4277.026063338116950934,8086.502208045348417539
...,...,...,...,...,...,...,...
1526,0xfee6be6b5cc8cb4ee8189850a69973e774e7614e,270.955512893611707508,-2382.961866524011759195,0,270.955512893611707508,-2366.961866524011759195,270.955512893611707508
1527,0xff5144b6d2aaa53eade50c308c23d5c2151582d2,-62.620033616776856488,-74.169385339819914922,1,-62.620033616776856488,-74.169385339819914922,-62.620033616776856488
1528,0xff6f84ce85166396d1402d14721ac350634a5d4b,1.208595124603055855,-3.47574880960140501,0,1.208595124603055855,-1.475748809601405010,1.208595124603055855
1529,0xff714698695caab8c83061bb9732967a50730570,-0.091948537070266261,-2.316814859350890134,0,-0.091948537070266261,-1.316814859350890134,-0.091948537070266261


In [50]:
df_compare[(df_compare['pnl_stat'] != df_compare['pnl_position'])]

,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl


In [51]:
df_compare[(df_compare['pnl_stat'] != df_compare['pnl'])]


,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl


## Get a specific position

In [52]:
position_id = '0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x7f'


In [53]:
df_trade[df_trade['positionId'] == position_id].sort_values('timestamp')


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl


In [54]:
df_position[df_position['id'] == position_id]


,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl


In [55]:
df_trade.loc[df_trade['positionId'] == position_id, 'netPnl'].sum()

0

## Get trader

In [56]:
trader = '0x1681Cad31E305874B5A21396E40FA7dC170B2791'.lower()


In [57]:
df_trade.loc[df_trade['account'] == trader, :].sort_values('timestamp')

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl
4474,0x3fbb627bdd898002889c7d2ce0444b2e29d6aa0a50a9...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677113535,OP,sOPPERP,DelayedOffchain,2.742483291565918274,12132.811078139668205447,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,227.367896457806152764,0,54784.386387060258102611,54784.386387060258102611,False,-227.367896457806152764
17520,0xfb2a1ff5159d78192c74cc8f1a5028aaf840d2095cf8...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677114963,OP,sOPPERP,DelayedOffchain,2.744877321366261343,19660.711270143107299557,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,227.675601935898782989,0,54811.338969317789061719,109595.72535637804716433,False,-227.675601935898782989
12537,0xb337cd37dd13c62361dacfb8d8c955ce0cffeba7359a...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677115555,OP,sOPPERP,DelayedOffchain,2.751664489714393578,20172.80702507197470106,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,303.570665687636824412,875.000091833533927965,-109595.72535637804716433,0,True,571.429426145897103553
8743,0x7cfdb28c36d0295505554b8565d1b134c5f335489d86...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677115723,OP,sOPPERP,DelayedOffchain,2.751838190521767399,19869.236359384337876648,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,453.501475148249907077,0,109381.788678168883665981,109381.788678168883665981,False,-453.501475148249907077
4342,0x3dc148b8d834a44e15c438907dfdeb7616f1c2638784...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677117568,OP,sOPPERP,DelayedOffchain,2.756007083249334791,20597.428467644389381177,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,11.017553470973167702,0,2181.308731214982746556,111563.097409383866412537,False,-11.017553470973167702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3037,0x2b480c12abb5df0e6a68d062e30fb880d8f105441a40...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677443741,OP,sOPPERP,DelayedOffchain,2.953902806694549706,188.108402537138835013,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x21e,2.008813435819985742,-0.173824774954700837,-1.989105351291291807,0,True,-2.182638210774686579
4056,0x39a9a365b6489eea28c5a08178b0e6118d036844bf02...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677446286,sETH,sETHPERP,DelayedOffchain,1648.182427311804008354,44555.16705323421869018,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xd26,249.227364096770601253,0,-300,-300,False,-249.227364096770601253
3212,0x2dcb38e39a368f98c0f3d30560655b52dc741609b505...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677446376,sETH,sETHPERP,DelayedOffchain,1648.182509171456408634,44305.591221217554198127,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xd26,125.613688187859230647,0,-150,-450,False,-125.613688187859230647
9216,0x840de87da938c3b8a79b83098413195351b5c92e28c4...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677458512,sETH,sETHPERP,DelayedOffchain,1636.055999288635818221,59318.245402543241515302,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xd26,83.802799964431790911,0,-50,-500,False,-83.802799964431790911


In [58]:
df_position.loc[df_position['account'] == trader, :].sort_values('openTimestamp')


,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl
5215,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,12132.811078139668205447,0,2.743680600844747876,2.743680600844747876,2.751664489714393578,2.751664489714393578,875.000091833533927965,...,758.614164081341760165,-8.649092571849980604,False,False,19871.236359384337876648,7628.688446064327502644,7628.688446064327502644,602266.331283440114914878,0xb337cd37dd13c62361dacfb8d8c955ce0cffeba7359a...,116.385927752192167800
5220,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,19869.236359384337876648,0,2.754837589211131778,2.754837589211131778,2.980229222807328583,2.980229222807328583,28770.194800749874879936,...,1004.370651815886884946,-752.138744990282332787,False,False,95384.491796858923487188,48499.570033530880051568,52862.570033530880051568,684936.196873252501376358,0x3cd7087e09a823862ed38ea4c89c5d3ec0d2de69150f...,27765.824148933987994990
5269,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x21e,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,95382.491796858923487188,0,3.041291227243444658,3.041291227243444658,2.953902806694549706,2.953902806694549706,-36484.850997685449922467,...,3448.634498715489397836,-3163.911932743271012393,False,False,188.099589101318849271,-52098.994778613393347069,104071.878138574616072575,2713643.9030200802085693,0x2b480c12abb5df0e6a68d062e30fb880d8f105441a40...,-39933.485496400939320303
4255,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xcca,0x1681cad31e305874b5a21396e40fa7dc170b2791,sETH,116168.872917188009419644,0,1630.118304795638683008,1630.118304795638683008,1646.57617530541193378,1646.57617530541193378,-39130.812539714651669238,...,12107.013580605613159124,-373.529045460285940088,False,False,58.064836561033330559,-64501.452914846425324233,3588.549712725441186117,15924289.640942608049046179,0x8a11a70fe3a2fe1e801ccfae29f1afaf7d0153e3979b...,-51237.826120320264828362
4354,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0xd26,0x1681cad31e305874b5a21396e40fa7dc170b2791,sETH,44555.16705323421869018,-1,1646.969809067382909424,1646.969809067382909424,1646.7498,NaN,-206.681059695158193416,...,1282.688468033380852806,-337.045796179240957438,False,True,65.869682847913594968,-42665.102055545908001326,9748.320090512447413872,1645529.520318010684708342,0x017cf9b778695d4b780da3316f7b270de178997c1346...,-1489.369527728539046222


## Plot market pnl over time

In [59]:
df_market_pnl = df_trade.sort_values(['asset', 'timestamp'])
df_market_pnl['netPnl'] = df_market_pnl['pnl'] - df_market_pnl['feesPaid']

df_market_pnl['timestamp'] = df_market_pnl['timestamp'].astype(np.int64)
df_market_pnl['pnl'] = df_market_pnl['pnl'].astype(np.float64)
df_market_pnl['netPnl'] = df_market_pnl['netPnl'].astype(np.float64)
df_market_pnl['feesPaid'] = df_market_pnl['feesPaid'].astype(np.float64)

df_market_pnl['pnl_cumulative'] = df_market_pnl.groupby('asset')['pnl'].cumsum()
df_market_pnl['feesPaid_cumulative'] = df_market_pnl.groupby('asset')['feesPaid'].cumsum()
df_market_pnl['netPnl_cumulative'] = df_market_pnl.groupby('asset')['netPnl'].cumsum()



In [60]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='pnl_cumulative',
    color='asset'
)

In [87]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='netPnl_cumulative',
    color='asset'
)


In [62]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='feesPaid_cumulative',
    color='asset'
)


In [63]:
df_market_pnl.to_csv('output/market_cumulative_pnl.csv', index=False)

In [64]:
df_market_pnl.groupby('asset')['netPnl'].sum().sort_values()

asset
sETH    -266549.747356
OP       -89923.546521
sBTC     -67880.880411
FTM      -11589.852245
DYDX      -4899.033432
FLOW      -3792.741814
UNI       -1217.175291
ATOM       -555.980876
EUR        -325.468393
AXS        -264.396845
GBP        -207.020084
AUD          -6.600175
XAG          -1.462378
XAU         953.675063
NEAR       1140.117507
AAVE       2782.282991
MATIC      3043.244354
LINK      10546.821482
DOGE      11998.512182
APE       13803.706772
SOL       31836.505947
BNB       41935.980888
AVAX      85509.800950
Name: netPnl, dtype: float64

## Do the same with position data

In [65]:
df_position_pnl = df_position[df_position['isOpen'] == False].sort_values(['asset', 'closeTimestamp'])
df_position_pnl['netPnl'] = df_position_pnl['pnl'] - df_position_pnl['feesPaid']

df_position_pnl['closeTimestamp'] = df_position_pnl['closeTimestamp'].astype(np.int64)
df_position_pnl['pnl'] = df_position_pnl['pnl'].astype(np.float64)
df_position_pnl['netPnl'] = df_position_pnl['netPnl'].astype(np.float64)
df_position_pnl['netFunding'] = df_position_pnl['netFunding'].astype(np.float64)

df_position_pnl['pnl_cumulative'] = df_position_pnl.groupby('asset')['pnl'].cumsum()
df_position_pnl['netPnl_cumulative'] = df_position_pnl.groupby('asset')['netPnl'].cumsum()
df_position_pnl['netFunding_cumulative'] = df_position_pnl.groupby('asset')['netFunding'].cumsum()



In [66]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='pnl_cumulative',
    color='asset'
)


In [78]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='netPnl_cumulative',
    color='asset'
)


In [79]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='netFunding_cumulative',
    color='asset'
)


In [80]:
df_position_pnl['netPnl'].sum()

-228158.8190688598

In [81]:
df_position_pnl[df_position_pnl['asset'] == 'ATOM']['closeTimestamp'].min()

1675894747